In [12]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [13]:
import numpy as np
import pandas as pd

In [14]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [15]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [16]:
# create an engine to connect to our database..
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [17]:
# reflect an existing database into a new model. to automap sql classes into python.
Base = automap_base()
# reflect the tables to only get python classes.
Base.prepare(engine, reflect=True)

In [18]:
# We can view all of the classes that automap found.
Base.classes.keys()

['measurement', 'station']

In [19]:
# Save references to each table by re-defining them.
Measurement = Base.classes.measurement
Station = Base.classes.station

In [20]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [21]:
# view columns for Measurment...
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2b3a9a36e08>,
 'prcp': 0.08,
 'date': '2010-01-01',
 'id': 1,
 'station': 'USC00519397',
 'tobs': 65.0}

In [22]:
# view columns for Station..
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2b3a9cf7c88>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'latitude': 21.2716,
 'id': 1,
 'station': 'USC00519397',
 'elevation': 3.0}

In [25]:
for row in session.query(Station.station, Station.name, Station.id).limit(15).all():
    print(row)

('USC00519397', 'WAIKIKI 717.2, HI US', 1)
('USC00513117', 'KANEOHE 838.1, HI US', 2)
('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 3)
('USC00517948', 'PEARL CITY, HI US', 4)
('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 5)
('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 6)
('USC00519281', 'WAIHEE 837.5, HI US', 7)
('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 8)
('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 9)


In [27]:
for row in session.query(Measurement.station, Measurement.prcp, Measurement.date, Measurement.tobs).limit(15).all():
    print(row)

('USC00519397', 0.08, '2010-01-01', 65.0)
('USC00519397', 0.0, '2010-01-02', 63.0)
('USC00519397', 0.0, '2010-01-03', 74.0)
('USC00519397', 0.0, '2010-01-04', 76.0)
('USC00519397', None, '2010-01-06', 73.0)
('USC00519397', 0.06, '2010-01-07', 70.0)
('USC00519397', 0.0, '2010-01-08', 64.0)
('USC00519397', 0.0, '2010-01-09', 68.0)
('USC00519397', 0.0, '2010-01-10', 73.0)
('USC00519397', 0.01, '2010-01-11', 64.0)
('USC00519397', 0.0, '2010-01-12', 61.0)
('USC00519397', 0.0, '2010-01-14', 66.0)
('USC00519397', 0.0, '2010-01-15', 65.0)
('USC00519397', 0.0, '2010-01-16', 68.0)
('USC00519397', 0.0, '2010-01-17', 64.0)


In [28]:
# first date in Measurement table..
session.query(Measurement.date).order_by(Measurement.date).first()

('2010-01-01')

In [30]:
# last date in Measurement table..
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

# Exploratory Climate Analysis

In [74]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date <= '2017-08-23').\
    filter(Measurement.date >= '2016-08-23').\
    order_by(Measurement.date).all()

data_df = pd.DataFrame(data, columns = ['date', 'prcp']).dropna()
data_df = data_df.groupby('date').mean()
data_df.tail()

,prcp
date,
2017-08-19,0.030000
2017-08-20,0.005000
2017-08-21,0.193333
2017-08-22,0.166667
2017-08-23,0.132500


In [75]:
data_df.plot(x= data_df.index, y = data_df['prcp'], kind='scatter')
plt.tight_layout()
plt.show()

KeyError: "None of [Index(['2016-08-23', '2016-08-24', '2016-08-25', '2016-08-26', '2016-08-27',\n       '2016-08-28', '2016-08-29', '2016-08-30', '2016-08-31', '2016-09-01',\n       ...\n       '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17', '2017-08-18',\n       '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23'],\n      dtype='object', name='date', length=366)] are in the [columns]"

In [39]:
# Calculate the date 1 year ago from the last data point in the database
query_date = dt.date(2017, 8, 23) - dt.timedelta(days= 365)
print(query_date)

2016-08-23


In [51]:
# Perform a query to retrieve the data and precipitation scores
# Measurement.id, Measurement.station, Measurement.tobs
data_query = session.query(Measurement.prcp, Measurement.date)

In [53]:
# Create a dataframe from the query results and set the index to the date column
# 'id', 'station', , 'tobs'
prcp_df = pd.DataFrame(data_query, columns = ['prcp', 'date'])
prcp_df = prcp_df.set_index('date')
prcp_df.tail()

,prcp
date,
2017-08-19,0.09
2017-08-20,NaN
2017-08-21,0.56
2017-08-22,0.50
2017-08-23,0.45


In [ ]:
# Sort the dataframe by date
prcp_df = prcp_df.sort_values(by='date', ascending=False)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
"""Note: what this means is to use the .plot() method on your dataframe variable,
and then format the labels with methods like plt.xlabel() and plt.title(). 

For example:
df.plot()
plt.xlabels()

As opposed to:
plt.bar()
plt.xlabels()

This is something you would never actually do. If you're importing matplotlib, you'd use
it to generate the plot, as well as the formatting. It's good practice, if you're importing a library,
to use it for everything you can use that library for. Every libarary you causes your program to take up
more memory, so use what you import.
"""

In [76]:
# Use Pandas to calcualte the summary statistics for the precipitation data
summary_stat = prcp_df.groupby(["date"])[["prcp"]].agg(['mean', 'median', 'var', 'std', 'sem'])
summary_stat

prcp                                     
                mean median       var       std       sem
date                                                     
2010-01-01  0.151429  0.150  0.005914  0.076904  0.029067
2010-01-02  0.002857  0.000  0.000057  0.007559  0.002857
2010-01-03  0.000000  0.000  0.000000  0.000000  0.000000
2010-01-04  0.001429  0.000  0.000014  0.003780  0.001429
2010-01-05  0.005000  0.000  0.000070  0.008367  0.003416
...              ...    ...       ...       ...       ...
2017-08-19  0.030000  0.000  0.002700  0.051962  0.030000
2017-08-20  0.005000  0.005  0.000050  0.007071  0.005000
2017-08-21  0.193333  0.020  0.100933  0.317700  0.183424
2017-08-22  0.166667  0.000  0.083333  0.288675  0.166667
2017-08-23  0.132500  0.040  0.046225  0.215000  0.107500

[2792 rows x 5 columns]

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
